## 示例：将AlexNet网络从pytorch转为onnx

In [1]:
import torch
import torchvision

In [47]:
# 创建模拟输入，并从torchvision中导入alexnet模型，并加载预训练的参数
dummy_input = torch.randn(10, 3, 244, 244, device='cuda')
model = torchvision.models.alexnet(pretrained=True).cuda()

设置模型图中输入输出的名字，仅用于显示，对模型没有其他影响。

网络的输入由简单的输入列表（即传递给forward（）方法的值）和参数的简单列表组成，可以指定部分名称。

In [45]:
input_names = ['actual_intput_1'] + ["learned_%d" % i for i in range(16)]
output_names = ['output1']

使用export 可以导出一个二进制文件，后缀是'onnx'。它保存有模型的结构和参数，将`verbose`设置为`True`可以将计算图打印出来，可以看到我们设置的名字。设置的`learned`指的是每一层的输入

In [50]:
torch.onnx.export(model, dummy_input, 'alexnet.onnx', verbose=True, input_names=input_names, output_names=output_names, dynamic_axes={'actual_intput_1': [0]})

graph(%actual_intput_1 : Float(10, 3, 244, 244),
      %learned_0 : Float(64, 3, 11, 11),
      %learned_1 : Float(64),
      %learned_2 : Float(192, 64, 5, 5),
      %learned_3 : Float(192),
      %learned_4 : Float(384, 192, 3, 3),
      %learned_5 : Float(384),
      %learned_6 : Float(256, 384, 3, 3),
      %learned_7 : Float(256),
      %learned_8 : Float(256, 256, 3, 3),
      %learned_9 : Float(256),
      %learned_10 : Float(4096, 9216),
      %learned_11 : Float(4096),
      %learned_12 : Float(4096, 4096),
      %learned_13 : Float(4096),
      %learned_14 : Float(1000, 4096),
      %learned_15 : Float(1000)):
  %17 : Float(10, 64, 60, 60) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[11, 11], pads=[2, 2, 2, 2], strides=[4, 4]](%actual_intput_1, %learned_0, %learned_1), scope: AlexNet/Sequential[features]/Conv2d[0] # /home/szq/anaconda3/envs/paddle/lib/python3.7/site-packages/torch/nn/modules/conv.py:340:0
  %18 : Float(10, 64, 60, 60) = onnx::Relu(%17), scope: AlexNe

In [51]:
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

## 跟踪和脚本
pytorch导出onnx有两种形式： trace 和 script

- trace 是根据输入运行一次模型，只导出和本次输入相关的运算。如模型针对不同的输入有不同的操作，则不能导出其他输入对应的操作。并且对于for和if这样的逻辑判断，也会被展开，逻辑判断中未执行的也不会被导出。 
- script 会导出一个ScriptModule，直接从pytorch代码导出。

In [10]:
import torch

In [11]:
class LoopModel(torch.nn.Module):
    def forward(self, x, y):
        for i in range(y):
            x = x + i
        return x 

In [12]:
model = LoopModel()
dummy_input = torch.ones(2, 3, dtype=torch.long)
loop_count = torch.tensor(5, dtype=torch.long)

In [13]:
dummy_input, loop_count

(tensor([[1, 1, 1],
         [1, 1, 1]]),
 tensor(5))

In [15]:
torch.onnx.export(model, (dummy_input, loop_count), 'loop.onnx', verbose=True, input_names=['x', 'y'])

graph(%x : Long(2, 3),
      %y : Long()):
  %2 : Tensor = onnx::Constant[value={1}]()
  %3 : Tensor = onnx::Add(%x, %2)
  %4 : Tensor = onnx::Constant[value={2}]()
  %5 : Tensor = onnx::Add(%3, %4)
  %6 : Tensor = onnx::Constant[value={3}]()
  %7 : Tensor = onnx::Add(%5, %6)
  %8 : Tensor = onnx::Constant[value={4}]()
  %9 : Long(2, 3) = onnx::Add(%7, %8)
  return (%9)



使用脚本导出来捕捉动态循环，则需要在script中写循环，并在`Module`中调用。

In [21]:
@torch.jit.script
def loop(x, y):
    for i in range(int(y)):
        x = x + i 
    return x 

In [22]:
class LoopModel2(torch.nn.Module):
    def forward(self, x, y):
        return loop(x, y)

In [23]:
model = LoopModel2()
dummy_input = torch.ones(2, 3, dtype=torch.long)
loop_count = torch.tensor(5, dtype=torch.long)

In [24]:
torch.onnx.export(model, (dummy_input, loop_count), 'loop.onnx', verbose=True, input_names=['input_data', 'loop_range'])

graph(%input_data : Long(2, 3),
      %loop_range : Long()):
  %2 : Long() = onnx::Constant[value={1}](), scope: LoopModel2/loop
  %3 : Tensor = onnx::Cast[to=9](%2)
  %4 : Long(2, 3) = onnx::Loop(%loop_range, %3, %input_data), scope: LoopModel2/loop # <ipython-input-21-d92b6cad4d7d>:3:5
    block0(%i.1 : Long(), %cond : bool, %x.6 : Long(2, 3)):
      %8 : Long(2, 3) = onnx::Add(%x.6, %i.1), scope: LoopModel2/loop # <ipython-input-21-d92b6cad4d7d>:4:13
      %9 : Tensor = onnx::Cast[to=9](%2)
      -> (%9, %8)
  return (%4)



通过上面的方式，动态循环被正确的捕捉了。 

In [26]:
import caffe2.python.onnx.backend as backend

In [31]:
import numpy as np
import onnx

In [28]:
model = onnx.load('loop.onnx')

In [32]:
rep = backend.prepare(model)
outputs = rep.run((dummy_input.numpy(), np.array(9).astype(np.int64)))

In [33]:
outputs

Outputs(_0=array([[37, 37, 37],
       [37, 37, 37]], dtype=int64))

In [34]:
import onnxruntime as ort
ort_sess = ort.InferenceSession('loop.onnx')
outputs = ort_sess.run(None, {'input_data': dummy_input.numpy(),
                              'loop_range': np.array(9).astype(np.int64)})
print(outputs)

[array([[37, 37, 37],
       [37, 37, 37]], dtype=int64)]


导出onnx的函数为：
> torch.onnx.export(model, args, f, export_params=True, verbose=False, training=TrainingMode.EVAL, input_names=None, output_names=None, aten=False, export_raw_ir=False, operator_export_type=None, opset_version=None, _retain_param_name=True, do_constant_folding=True, example_outputs=None, strip_doc_string=True, dynamic_axes=None, keep_initializers_as_inputs=None, custom_opsets=None, enable_onnx_checker=True, use_external_data_format=False)

所有参数：

- odel 要导出的参数  
- args `参数元组或张量`，模型输入参数列表，就是`forward`中需要的输入。所有非张量参数将硬编码到模型，其余的张量输入将作为参数，并且按照在args中的原有顺序排序。  
- f 输出文件的名字  
- export_params  默认为ture,是否导出模型参数，如果False将导出一个未训练的模型。参数的顺序和`model.state_dict().values()`相同
- verbose 默认为False，如果设置为True，将打印Debug信息，成功导出的话，打印的是计算图结构  
- training   
     - TrainingMode.EVAL 导出模型为推理模式  
     - TrainingMode.PRESERVE 如果model.training=False 则导出为推理，否则是训练   
     - TrainingMode.TRAINING 导出模型为训练模式  
- input_names 输入参数的名字，和输入参数顺序一一对应  
- output_names 输出的名字，与输出顺序一一对应  
- export_raw_ir和aten，默认False，不建议修改  
- operator_export_type 导出操作类型：  
    - OperatorExportTypes.ONNX: 所有操作作为常规ONNX操作导出(使用ONNX命名空间).  
    - OperatorExportTypes.ONNX_ATEN: 所有操作作为ATEN操作导出 (使用aten命名空间).  
    - OperatorExportTypes.ONNX_ATEN_FALLBACK：如果一个操作不被ONNX支持，或者它的symbolic丢失，作为aten操作导出
    - OperatorExportTypes.RAW: 导出 rawir
    - OperatorExportTypes.ONNX_FALLTHROUGH: If an op is not supported in ONNX, fall through and export the operator as is, as a custom ONNX op.
- opset_version 默认是9，导出到Onnx的操作版本，可以设置为11,新版本支持更多操作导出。
- do_constant_folding 默认为False，常量折叠优化。
- example_outputs 默认为None，模型的示例输出，导出一个ScriptModule或TorchScript函数时，必须提供.
- dynamic_axes 动态维度，使用字典指定输入输出的动态维度。`key`为输入输出的名字，`value`为将被设置为动态输入的维度索引。通常索引有两种方式指定：
    - 一个整数列表，指定动态输入的维度，然后会自动命名这些维度`dynamic_axes = {'input_1':[0, 2, 3],'input_2':[0],'output':[0, 1]}`
    - 使用一个字典， 为每个索引命名`dynamic_axes = {'input_1':{0:'batch',1:'width',2:'height'},'input_2':{0:'batch'},'output':{0:'batch',          1:'detections'}`
    - 也可以将上面两种方式结合
- keep_initializers_as_inputs 如果为True，则导出图形中的所有初始值设定项（通常对应于参数）也将作为输入添加到图形中。如果为False，则初始化器不会作为输入添加到图中，而只将非参数输入添加为输入。
- custom_opsets 在导出时指示自定义操作集域和版本的字典。
- enable_onnx_checker 如果为True，则onnx模型检查器将作为导出的一部分运行，以确保导出的模型是有效的onnx模型
- external_data_format 如果为True，则模型以ONNX外部数据格式导出，在这种情况下，一些模型参数存储在外部二进制文件中，而不是存储在ONNX模型文件本身中。有关格式详细信息，请参阅链接：